<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/LinearProgrammingPyomo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Programming in Pyomo

## Requirements

In [ ]:
# Install
!pip install pyomo
# Import
import os
import pyomo.environ as pe
import random

## Motivation Example

We want to maximize the energy production benefits of running 2 generators. The benefit of producing with generator 1 and 2 are 3&euro;/MWh and 5&euro;/MWh, respectively. The maximum capacity of generator 1 and 2 are 4MW and 6MW, respectively. Both generators share the same water cooling system. Generator 1 needs 3 units of water per MW, while generator 2 needs 2 units of water per MW. The maximum units of water available is 18.

This optimization model is formulated as follows:

$$
\begin{align}
\underset{x_1,x_2}{\max} \quad & 3x_1+5x_2 \\
\text{s.t.} \quad & 0 \leq x_1 \leq 4\\
& 0 \leq x_2 \leq 6\\
& 3x_1 + 2x_2 \leq 18
\end{align}
$$

Below we provide the Pyomo code to solve this optimization problem:

In [ ]:
# Model
m = pe.ConcreteModel()
# Variables
m.x1 = pe.Var(within=pe.NonNegativeReals,bounds=(0,4))
m.x2 = pe.Var(within=pe.NonNegativeReals,bounds=(0,6))
# Objective function
def obj_rule(m):
  return 3*m.x1 + 5*m.x2
m.obj = pe.Objective(rule=obj_rule,sense=pe.maximize)
# Constraints
def con_rule(m):
  return 3*m.x1 + 2*m.x2 <= 18
m.con = pe.Constraint(rule=con_rule)
# Solve problem using NEOS server
os.environ['NEOS_EMAIL'] = 'xxx@gmail.com'
res = pe.SolverManagerFactory('neos').solve(m,opt='cplex')
# Print results
m.pprint()
print(res['Solver'][0])
print('x1 =',m.x1.value)
print('x2 =',m.x2.value)
print('Optimal value =',m.obj())

## General LP

We create a function to solve a general linear programming problem formulated below

$$
\begin{align}
\underset{x\geq 0}{\min} \quad & c^Tx \\
\text{s.t.} \quad & Ax \leq b\
\end{align}
$$

where $x\in\mathbb{R}^n$, $c\in\mathbb{R}^n$, $A\in\mathbb{R}^{m\times n}$, $b\in\mathbb{R}^m$. Elements of $A,b,c$ are randomly generating using standard normal distributions.

In [ ]:
def lp(nvar=10,ncon=5):
  # Random vectors and matrix of linear problem
  c = [round(abs(random.gauss(0,1)),2) for i in range(nvar)]
  A = [[round(random.gauss(0,1),2) for i in range(nvar)] for j in range(ncon)]
  b = [round(random.gauss(0,1),2) for j in range(ncon)]
  # Model
  m = pe.ConcreteModel()
  # Sets
  m.i = pe.Set(initialize=range(nvar),ordered=True)
  m.j = pe.Set(initialize=range(ncon),ordered=True)
  # Variables
  m.z = pe.Var()
  m.x = pe.Var(m.i,within=pe.NonNegativeReals)
  # Objective function
  def obj_rule(m):
    return sum(c[i]*m.x[i] for i in m.i)
  m.obj = pe.Objective(rule=obj_rule)
  # Constraints
  def con_rule(m,j):
    return sum(A[j][i]*m.x[i] for i in m.i) <= b[j]
  m.con = pe.Constraint(m.j,rule=con_rule)
  # Solve problem using NEOS server
  os.environ['NEOS_EMAIL'] = 'xxx@gmail.com'
  res = pe.SolverManagerFactory('neos').solve(m,opt=pe.SolverFactory('cplex'))
  print(res)

lp(nvar=10,ncon=5)